# Generative networks

Recurrent Neural Networks (RNNs) and their gated cell variants such as Long Short-Term Memory cells (LSTMs) and Gated Recurrent Units (GRUs) provide a mechanism for language modeling &mdash; they can learn word ordering and make predictions for the next word in a sequence. This allows us to use RNNs for **generative tasks**, such as ordinary text generation, machine translation, and even image captioning.

In the RNN architecture we discussed in the previous notebook, each RNN unit produced the next hidden state as an output. However, we could also add another output to each recurrent unit, which would allow us to output a **sequence** with the same length as the original sequence. Moreover, we could use RNN units that don't accept an input at each step, instead they take an initial state vector and produce a sequence of outputs.

This allows for different neural architectures, as is shown in the picture below:

![Image showing common recurrent neural network patterns.](notebooks/images/unreasonable-effectiveness-of-rnn.jpg)
*Image from blog post [Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by [Andrej Karpaty](http://karpathy.github.io/)*

* **One-to-one** is a traditional neural network with one input and one output.
* **One-to-many** is a generative architecture that accepts one input value, and generates a sequence of output values. For example, if we want to train an **image captioning** network that produces a textual description of a picture, we can use a picture as input, pass it through a CNN to obtain the hidden state, and then have a recurrent chain generate the caption word-by-word.
* **Many-to-one** corresponds to the RNN architectures we described in the previous unit, such as text classification.
* **Many-to-many**, or **sequence-to-sequence** corresponds to tasks such as **machine translation**, in which case the RNN first collects all the information from the input sequence into the hidden state, and then unrolls this state into the output sequence.

In this unit, we will focus on simple generative models that help us generate text. For simplicity, let's build a **character-level network**, which generates text letter by letter. During training, we need to take a text corpus, and split it into letter sequences. 

> For the sandbox environment, we need to run the following cell to make sure the required library is installed, and data is prefetched. If you're running locally, you can skip the following cell.

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

/Users/sergio/Library/Caches/pypoetry/virtualenvs/intro-tensorflow-ipOXWJBT-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Building character vocabulary

To build a character-level generative network, we need to split the text into individual characters instead of words. The `TextVectorization` layer that we've been using can't do that, so we have two options:

* Manually load text and implement our own tokenization, as in [this official Keras example](https://keras.io/examples/generative/lstm_character_level_text_generation/).
* Use the `Tokenizer` class for character-level tokenization.

We'll go with the second option. `Tokenizer` can also be used to tokenize into words, so we should be able to switch from character-level to word-level tokenization quite easily.

To do character-level tokenization, we need to pass the `char_level=True` parameter:

In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

2024-04-30 02:39:16.698057: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


We also want to use one special token to denote **end-of-sequence**, which we'll call `<eos>`. Let's add it manually to the vocabulary:

In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Now, to encode text into sequences of numbers, we can use:

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Training a generative RNN to generate titles

Here's how we'll train an RNN to generate titles: on each step, we'll take one title as input, and for each input character in that title, we'll train the network to generate the next character as output:

![Image showing an example RNN generation of the word 'HELLO'.](notebooks/images/rnn-generate.png)

For the last character of our sequence, we'll ask the network to generate the `<eos>` token.

The main difference between the generative RNN we're using here and the RNNs we've seen previously is that each cell of the generative RNN will produce an output, not just the final cell. This can be achieved by specifying the `return_sequences` parameter of the RNN cell.

Thus, during training, an input to the network would be a sequence of encoded characters of some length, and the corresponding output would be a sequence of the same length, but shifted by one element and terminated by `<eos>`. The minibatch will consist of several such sequences, and we'll need to use **padding** to align all sequences.

Let's create functions that transform the dataset for us. Because we want to pad sequences at the minibatch level, we'll first create the minibatches by calling `.batch()`, and then use `map` to do transformation. Here's the code for the function that does the transformation, which takes a whole minibatch as a parameter:

In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

A few important things that we do here:
* We first extract the actual text from the string tensor.
* `text_to_sequences` converts the list of strings into a list of integer tensors.
* `pad_sequences` pads those tensors to their maximum length.
* We finally one-hot encode all the characters, and also do the shifting and `<eos>` appending. We will soon see why we need one-hot-encoded characters.

However, this function is **Pythonic** &mdash; it cannot be automatically translated into the TensorFlow computational graph. (Computational graphs will be discussed in more detail in the module on going beyond Keras of this learning path.) If we try to pass it directly to the `Dataset.map` function, which is how we intend to use this function, we'll get an error. To get around this problem, we need to enclose this Pythonic call by using the `py_function` wrapper: 

In [33]:
def title_batch_fn(x):
    x = x['title']
    input_shape = x.shape+x.shape
    print(input_shape)
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    a.set_shape(input_shape+(vocab_size,))
    b.set_shape(input_shape+(vocab_size,))
    return a,b

> **Note**: You may be wondering if instead of calling the `py_function` wrapper we could transform the dataset using standard Python functions before passing it to `fit`. While this can definitely be done, when using `Dataset.map` the data transformation pipeline is executed using TensorFlow's computational graph, which takes advantage of GPU computations and minimizes the need to pass data between CPU/GPU.

We can now build our generator network and start training. Our network can be based on any recurrent cell which we discussed in the previous unit (simple, LSTM or GRU). In our example we'll use LSTM.

Because the network takes characters as input, the vocabulary size is pretty small. Therefore we don't need an embedding layer &mdash; we can feed one-hot-encoded input directly into the LSTM cell. The output layer is then a `Dense` classifier that will convert the LSTM output into one-hot-encoded token numbers.

In addition, since we're dealing with variable-length sequences, we can use a `Masking` layer to create a mask that will ignore the padded part of the string. This is not strictly needed, because we're not interested in everything that goes beyond the `<eos>` token, but we will use it to get some experience with this layer type. The input shape is `(None, vocab_size)`, where `None` indicates the sequence of variable length, and the output shape is also `(None, vocab_size)`, as you can see from the `summary` printout below: 

In [34]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_14 (Masking)            │ (None, None, 84)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, None, 128)      │       109,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, None, 84)       │        10,836 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,892 (468.33 KB)

 Trainable params: 119,892 (468.33 KB)

 Non-trainable params: 0 (0.00 B)

(None, None)
15000/15000 ━━━━━━━━━━━━━━━━━━━━ 341s 23ms/step - loss: 1.8569


## Generating output

Now that we've trained the model, we want to use it to generate output. First of all, we need a way to decode text represented by a sequence of token numbers. We could use the `tokenizer.sequences_to_texts` function, but it doesn't work well with character-level tokenization. Therefore we will take a dictionary of tokens from the tokenizer (called `word_index`), build a reverse map, and write our own decoding function:

In [35]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map.get(t,'') for t in x])

Now, let's do the generation. We first encode a string passed as parameter into a sequence, and then on each step we call our network to infer the next character. 

The output of the network is a vector of `vocab_size` elements representing probablities of each token, and we can find the most probably token number by using `argmax`. We then append this character to the generated list of tokens, and proceed with the generation. This process of generating one character is repeated `size` times to generate the required number of characters, and we terminate early if `eos_token` is encountered.

In [36]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;Black #39;s set to count of the the start of the man #39;'

## Sampling output during training

Because we don't have any useful metrics such as *accuracy*, the only way we can see that our model is getting better is by **sampling** generated strings during training. We use **callbacks** for that, which are functions that we can pass to the `fit` function, and that will be called periodically during training. 

In [38]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

(None, None)
Epoch 1/3


15000/15000 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.2457Today #39;; #39;s new the the the plan to contract in Iraq
15000/15000 ━━━━━━━━━━━━━━━━━━━━ 353s 24ms/step - loss: 1.2457
Epoch 2/3
14999/15000 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.2172Today #39;dotering to the the the the the plan #39;
15000/15000 ━━━━━━━━━━━━━━━━━━━━ 346s 23ms/step - loss: 1.2172
Epoch 3/3
15000/15000 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.1965Today #39;dotersured to the Martha #39; to start on the security for the security
15000/15000 ━━━━━━━━━━━━━━━━━━━━ 365s 24ms/step - loss: 1.1965


This example already generates some pretty good text, but it can be further improved in several ways:
* **More text**. We only used titles for our task, but you may want to experiment with full text. Remember that RNNs aren't that great at handling long sequences, so it makes sense to either split them into shorter sentences, or to always train on a sequence of fixed length `num_chars` (say, 256). You may try to change the example above into this architecture, using the [official Keras tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/) as inspiration.
* **Multilayer LSTM**. It makes sense to try 2 or 3 layers of LSTM cells. As we mentioned in the previous unit, each layer of LSTM extracts certain patterns from the text, and in the case of a character-level generator we can expect lower LSTM levels to be responsible for extracting syllables, and higher levels to extract words and word combinations. This can be implemented by adding layers sequentially to the model.
* You may also want to experiment with **GRU units** and see which one performs better, and with **different hidden layer sizes**. A hidden layer that's too large may result in overfitting (because the network will learn the exact text), and a hidden layer that's too small might not produce good results.

## Soft text generation and temperature

In the code for the `generate` function, we took the character with the highest probability as the next character in the generated text. This resulted in text that cycles between the same character sequences again and again, like in this example:
```
today of the second the company and a second the company ...
```

However, if we look at the probability distribution for the next character, it may be that there are several high probabilities that are pretty similar. For example, when looking for the next character in the sequence '*play*', it's similarly likely that it's either space or **e** (as in the word *player*).

Therefore, it's not always the best choice to select the character with the absolute highest probability &mdash; choosing the second or third highest might still lead to meaningful text, and may avoid cycling through character sequences. Therefore, a better strategy is to **sample** characters from the probability distribution given by the network output.

This sampling can be done using the `np.multinomial` function which implements a **multinomial distribution**. A function that implements this **soft** text generation is defined below:

In [39]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;dotersured #39; Afghan start to be on the rest
On Sunday sarcts of the peace of the second at &lt;b&gt;...&lt;/b&gt;
Moscow, S suit for \$100M in the profit in Iraq attack (AFP)
President t at Blair #39;s end the plan in Contract of Thatches
Little red riding hood foribal for profit in research &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today U Agents Free Bryant as Hell at Pregise Retail World'
On Sunday C Fallujah Wored Peace Pleave Volzertion (AP)
Moscow, RIIN Adote State Slidect Rule Plan to reported at game
President RIC Warns Most It to Higherty Too Attacks (Reuters)
Little red riding hood uditual - E-M betale sales

--- Temperature = 1.0
Today (ReneWractions in Break of a flight  #39;Oracle #39; by inclociady
On Sunday G bunder a quot; up will Lity
Moscow, ravist rebels digman die in about Davey profit
President WC Makes Piloties to MetJ4 on Progress to Station (AP)
Little red riding hood for bankle; ancidate to tax cric Note

--- Temperature 

We've introduced one more parameter called **temperature**, which indicates how strongly we should choose higher probability characters over lower probability ones. If the temperature is close to 0, we choose the highest probability character, and when the temperature approaches infinity then all probabilities become equal, and we randomly select the next character. In the example above we can observe that the output becomes meaningless when we increase the temperature too much, and starts cycling when we lower it closer to 0. 